In [3]:
!pip install --upgrade scipy numpy


     |████████████████████████████████| 34.5 MB 1.5 MB/s eta 0:00:01
Requirement already up-to-date: numpy in /opt/conda/lib/python3.8/site-packages (1.24.4)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.1
    Uninstalling scipy-1.7.1:
      Successfully uninstalled scipy-1.7.1


In [7]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader
from load_data import *
import pandas as pd
import torch
import torch.nn.functional as F

import pickle as pickle
import numpy as np
import argparse
from tqdm import tqdm

from train import set_seed

dir_name = 'train_19_roberta-large8379'
model_dir = f"../code/results/{dir_name}"
def inference(model, tokenized_sent, device):
  """
    test dataset을 DataLoader로 만들어 준 후,
    batch_size로 나눠 model이 예측 합니다.
  """
  dataloader = DataLoader(tokenized_sent, batch_size=16, shuffle=False)
  model.eval()
  output_pred = []
  output_prob = []
  for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
      outputs = model(
          input_ids=data['input_ids'].to(device),
          attention_mask=data['attention_mask'].to(device),
          token_type_ids=data['token_type_ids'].to(device)
          )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
  return np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

def num_to_label(label):
  """
    숫자로 되어 있던 class를 원본 문자열 라벨로 변환 합니다.
  """
  origin_label = []
  with open('../code/dict_num_to_label.pkl', 'rb') as f:
    dict_num_to_label = pickle.load(f)
  for v in label:
    origin_label.append(dict_num_to_label[v])
  
  return origin_label

def load_test_dataset(dataset_dir, tokenizer):
  """
    test dataset을 불러온 후,
    tokenizing 합니다.
  """
  test_dataset = load_data(dataset_dir)
  test_label = list(map(int,test_dataset['label'].values))
  # tokenizing dataset
  tokenized_test = tokenized_dataset19(test_dataset, tokenizer)
  return test_dataset['id'], tokenized_test, test_label

def main():
  set_seed(42)
  """
    주어진 dataset csv 파일과 같은 형태일 경우 inference 가능한 코드입니다.
  """
#   device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  device = torch.device('cpu')
  # load tokenizer
  # Tokenizer_NAME = "klue/bert-base"
  Tokenizer_NAME = "klue/roberta-large"

  tokenizer = AutoTokenizer.from_pretrained(Tokenizer_NAME)
  tokenizer.add_special_tokens({ "additional_special_tokens": ['<PER>', '<ORG>', '<DAT>', '<LOC>', '<POH>', '<NOH>','<s>','</s>','<o>','</o>',
                                                               '<S. PER>', '<S. ORG>', '<S. DAT>', '<S. LOC>', '<S. POH>', '<S. NOH>',
                                                               '<O. PER>', '<O. ORG>', '<O. DAT>', '<O. LOC>', '<O. POH>', '<O. NOH>']})

  file_list = os.listdir(model_dir)
  
  ## load test datset
  test_dataset_dir = "../data/dataset/test/test_data.csv"
  test_id, test_dataset, test_label = load_test_dataset(test_dataset_dir, tokenizer)
  Re_test_dataset = RE_Dataset(test_dataset ,test_label)
  print(file_list)
  for file in file_list:
    print(file)
    # if os.path.exists(f'./code/prediction/{dir_name}_{file}.csv'):
    #   continue
    ## load my model
    MODEL_NAME = f'{model_dir}/{file}' # model dir.
    print(MODEL_NAME)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
    model.parameters
    model.to(device)
    model.resize_token_embeddings(len(tokenizer))


    ## predict answer
    pred_answer, output_prob = inference(model, Re_test_dataset, device) # model에서 class 추론
    pred_answer = num_to_label(pred_answer) # 숫자로 된 class를 원래 문자열 라벨로 변환.
    
    ## make csv file with predicted answer
    #########################################################
    # 아래 directory와 columns의 형태는 지켜주시기 바랍니다.
    output = pd.DataFrame({'id':test_id,'pred_label':pred_answer,'probs':output_prob})

    output.to_csv(f'../code/prediction/{dir_name}_{file}.csv', index=False) # 최종적으로 완성된 예측한 라벨 csv 파일 형태로 저장.
  #### 필수!! ##############################################
  print('---- Finish! ----')
# if __name__ == '__main__':
parser = argparse.ArgumentParser()

# model dir
# parser.add_argument('--model_dir', type=str, default=f"./best_model")
# parser.add_argument('--model_dir', type=str, default=f"../code/results/{dir_name}")

# args = parser.parse_args()
# print(args)
main()
  


['checkpoint-4000', 'checkpoint-7000', 'checkpoint-6000', 'checkpoint-5000']
checkpoint-4000
../code/results/train_19_roberta-large8379/checkpoint-4000


  1%|          | 3/486 [01:14<3:21:03, 24.98s/it]


KeyboardInterrupt: 